In [1]:
from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

# Display progress logs on stdout
#import logging
#logging.basicConfig(level=logging.DEBUG,
#                    format='%(asctime)s %(levelname)s %(message)s')

from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
basedir="/home/tutysara/src/myprojects/dog-project/dogImages"
train_idx_path = basedir+ "/train_list.txt"
valid_idx_path = basedir+ "/valid_list.txt"
test_idx_path = basedir+ "/test_list.txt"


col_names = ["X", "y"]
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
train_data_df.y = train_data_df.y-1
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
valid_data_df.y = valid_data_df.y-1
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)
test_data_df.y = test_data_df.y-1

In [3]:
test_data_df.head()

,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [4]:
print(train_data_df.shape)
print(valid_data_df.shape)
print(test_data_df.shape)

print(train_data_df.shape[0] + valid_data_df.shape[0] + test_data_df.shape[0])
sample_df = test_data_df[:5]
sample_df

(6680, 2)
(835, 2)
(836, 2)
8351


,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [38]:
# load TTD
ttd_basedir="/home/tutysara/src/myprojects/senti/dataset"
ttd_data_idx_path = ttd_basedir+ "/twitter_five_agrees.txt"
col_names = ["X", "y"]
ttd_data_df = pd.read_csv(ttd_data_idx_path, sep=" ", header=None, names=col_names)
ttd_data_df.X = ttd_data_df.X.apply(lambda x: ttd_basedir+"/Agg_AMT_Candidates/"+x)
print(ttd_data_df.shape)
ttd_data_df.head()

(882, 2)


,X,y
0,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/28800526...,0
1,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/33547480...,1
2,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/27176682...,1
3,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/10096570...,1
4,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/13247920...,1


In [6]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [7]:
#definition of data generator
def df_data_generator(df, batch_size=32, num_classes=3, shuffle=False, basedir=".", transformer=None):    
    while True:
        if shuffle:
            df = df.sample(frac=1)
            
        X_file_name = df.X.apply(lambda x: basedir+"/"+x) 
        y = np_utils.to_categorical(df.y, num_classes)
    # infinitely serve batches
        max_range = (1+ df.shape[0]//batch_size)
        for i in range(max_range):
            if transformer:
                yield transformer(X_file_name[i*batch_size : (i+1)*batch_size]).astype('float32'), y[i*batch_size : (i+1)*batch_size]
            else:
                yield X_file_name[i*batch_size : (i+1)*batch_size], y[i*batch_size : (i+1)*batch_size]
                

from keras.applications.mobilenet import MobileNet
import keras.applications.mobilenet as mobilenet
def img_to_mobilenet_tensor_transformer(x):
    return mobilenet.preprocess_input(paths_to_tensor(x))


In [10]:
import bcolz
def bcolz_writer(gen, steps, dirname ):
    X = None; y = None
    data_dir = dirname + "_data.bclz"
    label_dir = dirname + "_labels.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        if i== 0:
            X = bcolz.carray(X_batch, rootdir=data_dir, mode='w')
            y = bcolz.carray(y_batch, rootdir=label_dir, mode='w') 
        else:
            X.append(X_batch)
            y.append(y_batch)
    X.flush()
    y.flush()
    return X, y

In [8]:
bnf_train_name = 'bottleneck_features_mobilenet_train'
bnf_valid_name = 'bottleneck_features_mobilenet_valid'
bnf_test_name = 'bottleneck_features_mobilenet_test' 

In [9]:
mobilenet_train_name = 'mobilenet_train_data'
mobilenet_valid_name = 'mobilenet_valid_data'
mobilenet_test_name = 'mobilenet_test_data'

In [11]:
## load train, test, and validation datasets
## load and convert to bcolz format for fast reading

import time

batch_size=256
num_classes = 133
train_gen = df_data_generator(train_data_df,
                              batch_size=batch_size, 
                              transformer=img_to_mobilenet_tensor_transformer, 
                              basedir=basedir,
                              num_classes=num_classes)

valid_gen = df_data_generator(valid_data_df,
                              batch_size=batch_size, 
                              transformer=img_to_mobilenet_tensor_transformer, 
                              basedir=basedir,
                              num_classes=num_classes)

test_gen = df_data_generator(test_data_df,
                             batch_size=batch_size, 
                             transformer=img_to_mobilenet_tensor_transformer, 
                             basedir=basedir,
                             num_classes=num_classes)

s= time.time()
!rm -rf 'mobilenet_valid_data*bclz'
mobilenet_valid_data,  mobilenet_valid_labels = bcolz_writer(gen=valid_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        dirname=mobilenet_valid_name)
print("Took {} seconds to calculate mobilenet_valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'mobilenet_test_data*bclz'
mobilenet_test_data,  mobilenet_test_labels = bcolz_writer(gen=test_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        dirname=mobilenet_test_name)
print("Took {} seconds to calculate mobilenet_test_data".format(time.time()-s))

s= time.time()
!rm -rf 'mobilenet_train_data*bclz'
mobilenet_train_data,  mobilenet_train_labels = bcolz_writer(gen=train_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        dirname=mobilenet_train_name)
print("Took {} seconds to calculate mobilenet_train_data".format(time.time()-s))

Took 9.897374153137207 seconds to calculate mobilenet_valid_data
Took 9.92434811592102 seconds to calculate mobilenet_test_data
Took 89.60999917984009 seconds to calculate mobilenet_train_data


In [41]:
print(mobilenet_valid_data.shape)
print(mobilenet_test_data.shape)
print(mobilenet_train_data.shape)

print(mobilenet_valid_labels.shape)
print(mobilenet_test_labels.shape)
print(mobilenet_train_labels.shape)

(835, 224, 224, 3)
(836, 224, 224, 3)
(6680, 224, 224, 3)
(835, 133)
(836, 133)
(6680, 133)


In [44]:
# read from disk and check size
mobilenet_valid_data = bcolz.carray(rootdir= mobilenet_valid_name+'_data.bclz', mode='r')
mobilenet_test_data = bcolz.carray(rootdir= mobilenet_test_name + '_data.bclz', mode='r')
mobilenet_train_data = bcolz.carray(rootdir= mobilenet_train_name+ '_data.bclz', mode='r')


mobilenet_valid_labels = bcolz.carray(rootdir= mobilenet_valid_name+'_labels.bclz', mode='r')
mobilenet_test_labels = bcolz.carray(rootdir= mobilenet_test_name + '_labels.bclz', mode='r')
mobilenet_train_labels = bcolz.carray(rootdir= mobilenet_train_name+ '_labels.bclz', mode='r')

print(mobilenet_valid_data.shape)
print(mobilenet_test_data.shape)
print(mobilenet_train_data.shape)

print(mobilenet_valid_labels.shape)
print(mobilenet_test_labels.shape)
print(mobilenet_train_labels.shape)

(835, 224, 224, 3)
(836, 224, 224, 3)
(6680, 224, 224, 3)
(835, 133)
(836, 133)
(6680, 133)


In [13]:
def bcolz_prediction_writer(gen, steps, model, dirname ):
    X = None; y = None
    data_dir = dirname + "_data.bclz"
    label_dir = dirname + "_labels.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        X_out = model.predict(X_batch)
        if i== 0:
            X = bcolz.carray(X_out, rootdir=data_dir, mode='w')
            y = bcolz.carray(y_batch, rootdir=label_dir, mode='w') 
        else:
            X.append(X_out)
            y.append(y_batch)
    X.flush()
    y.flush()
    return X, y

In [14]:
def bnf_data_generator(bnf_data, bnf_labels, batch_size=32):
    while True:
        max_range = (1 + bnf_data.shape[0]//batch_size)
        for i in range(max_range):
            yield bnf_data[i*batch_size : (i+1)*batch_size], bnf_labels[i*batch_size : (i+1)*batch_size]                

In [15]:
## make a generator of loaded data
batch_size = 256
mobilenet_valid_gen = bnf_data_generator(mobilenet_valid_data, mobilenet_valid_labels, batch_size=batch_size)
mobilenet_test_gen = bnf_data_generator(mobilenet_test_data, mobilenet_test_labels, batch_size=batch_size)
mobilenet_train_gen = bnf_data_generator(mobilenet_train_data, mobilenet_train_labels, batch_size=batch_size)

In [16]:
## Transform data and save it in bottleneck feacture format

from keras.applications.mobilenet import MobileNet
import time

mobilenet_feature_ext = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
mobilenet_feature_ext._make_predict_function()

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_valid*bclz'
bnf_valid_data,  bnf_valid_labels = bcolz_prediction_writer(gen=mobilenet_valid_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_valid_name)
print("Took {} seconds to calculate bnf_valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_test*bclz'
bnf_test_data,  bnf_test_labels = bcolz_prediction_writer(gen=mobilenet_test_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_test_name)
print("Took {} seconds to calculate bnf_test_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_train*bclz'
bnf_train_data,  bnf_train_labels = bcolz_prediction_writer(gen=mobilenet_train_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_train_name)
print("Took {} seconds to calculate bottleneck_features_mobilenet_train".format(time.time()-s))


Took 4.822598457336426 seconds to calculate bnf_valid_data
Took 3.8090832233428955 seconds to calculate bnf_test_data
Took 28.038143157958984 seconds to calculate bottleneck_features_mobilenet_train


In [45]:
print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)

print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(835, 7, 7, 1024)
(836, 7, 7, 1024)
(6680, 7, 7, 1024)
(835, 133)
(836, 133)
(6680, 133)


In [46]:
## Read it back from disk and check size
bnf_valid_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_data.bclz', mode='r')
bnf_test_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_data.bclz', mode='r')
bnf_train_data = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_data.bclz', mode='r')

bnf_valid_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_valid_labels.bclz', mode='r')
bnf_test_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_test_labels.bclz', mode='r')
bnf_train_labels = bcolz.carray(rootdir='bottleneck_features_mobilenet_train_labels.bclz', mode='r')

print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)


print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(835, 7, 7, 1024)
(836, 7, 7, 1024)
(6680, 7, 7, 1024)
(835, 133)
(836, 133)
(6680, 133)


In [21]:
## make a generator of loaded data
batch_size = 256
train_gen_fex =bnf_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
valid_gen_fex =bnf_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
test_gen_fex = bnf_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size) 

In [47]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras import regularizers

alpha = 1
num_classes = 133
dropout=1e-3

if K.image_data_format() == 'channels_first':
    shape = (int(1024 * alpha), 1, 1)
else:
    shape = (1, 1, int(1024 * alpha))

classes = num_classes

top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 1024)))
top_model.add(Reshape(shape, name='reshape_1'))
top_model.add(Dropout(dropout, name='dropout'))
top_model.add(Conv2D(classes, (1, 1),
           padding='same', name='conv_preds'))
top_model.add(Activation('softmax', name='act_softmax'))
top_model.add(Reshape((classes,), name='reshape_2'))

top_model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1024)        0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 133)         136325    
_________________________________________________________________
act_softmax (Activation)     (None, 1, 1, 133)         0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 133)               0         
Total params: 136,325
Trainable params: 136,325
Non-trainable params: 0
_________________________________________________________________


In [23]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
top_model.fit_generator(train_gen_fex,
          steps_per_epoch= (1 + (train_data_df.shape[0]// batch_size)),
          epochs=25,
          validation_data=valid_gen_fex,
          validation_steps= (1 + (valid_data_df.shape[0] // batch_size)),
          callbacks=[early_stopping, checkpointer])

Epoch 1/25
27/27 [==============================] - 3s 106ms/step - loss: 8.2892 - acc: 0.0010 - val_loss: 5.3629 - val_acc: 0.0347
Epoch 2/25
27/27 [==============================] - 2s 85ms/step - loss: 4.7870 - acc: 0.0281 - val_loss: 3.9164 - val_acc: 0.1317
Epoch 3/25
27/27 [==============================] - 2s 87ms/step - loss: 4.0677 - acc: 0.0809 - val_loss: 3.1081 - val_acc: 0.2802
Epoch 4/25
27/27 [==============================] - 2s 85ms/step - loss: 3.2001 - acc: 0.2199 - val_loss: 2.5705 - val_acc: 0.3988
Epoch 5/25
27/27 [==============================] - 2s 91ms/step - loss: 2.6737 - acc: 0.3287 - val_loss: 2.1741 - val_acc: 0.4802
Epoch 6/25
27/27 [==============================] - 2s 88ms/step - loss: 2.2352 - acc: 0.4413 - val_loss: 1.8844 - val_acc: 0.5557
Epoch 7/25
27/27 [==============================] - 2s 88ms/step - loss: 1.9055 - acc: 0.5305 - val_loss: 1.6674 - val_acc: 0.6060
Epoch 8/25
27/27 [==============================] - 2s 85ms/step - loss: 1.6454 - 

In [24]:
def prediction_from_gen(gen, steps, model, dirname ):
    y_true = None; y_pred = None
    yt_dir = dirname + "_y_true.bclz"
    yp_dir = dirname + "_y_pred.bclz"
    for i in range(steps):
        X_batch, y_batch = next(gen)
        y_out = model.predict(X_batch)
        if i== 0:
            y_true = bcolz.carray(y_batch, rootdir=yt_dir, mode='w')
            y_pred = bcolz.carray(y_out, rootdir=yp_dir, mode='w') 
        else:
            y_true.append(y_batch)
            y_pred.append(y_out)
    y_true.flush()
    y_pred.flush()
    return y_true, y_pred

y_true, y_pred = prediction_from_gen(gen=test_gen_fex,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=top_model,
                                     dirname="bottleneck_features_mobilenet_test"
                                    )


In [25]:
print(y_true.shape)
print(y_pred.shape)

(836, 133)
(836, 133)


In [26]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 77.9904%


In [27]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet.preprocess_input(paths_to_tensor(ttd_X1))
ttd_X = mobilenet_feature_ext.predict(ttd_X2)

In [28]:
ttd_y_pred = top_model.predict(ttd_X)
ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [29]:
print(ttd_X.shape)
print(ttd_y_pred.shape)
print(ttd_y_true.shape) 

(882, 7, 7, 1024)
(882, 133)
(882,)


In [30]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 59.2971%


In [31]:
# fine tune on full model
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

mobilenet_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# CREATE AN "REAL" MODEL FROM Mobilenet
# BY COPYING ALL THE LAYERS OF Mobilenet
model = Sequential()
for l in mobilenet_model.layers:
    model.add(l)


# CONCATENATE THE TWO MODELS
model.add(top_model)

# LOCK THE TOP CONV LAYERS
for layer in model.layers:
    layer.trainable = False

# COMPILE THE MODEL
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [32]:
# load train, test, and validation datasets
batch_size=256

from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fullmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit_generator(mobilenet_train_gen,
          steps_per_epoch= (1+ train_data_df.shape[0]// batch_size),
          epochs=2,
          validation_data=mobilenet_valid_gen,
          validation_steps= (1+ valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

Epoch 1/2
27/27 [==============================] - 23s 836ms/step - loss: 1.5138 - acc: 0.6198 - val_loss: 0.7580 - val_acc: 0.7880
Epoch 2/2
27/27 [==============================] - 18s 672ms/step - loss: 1.5131 - acc: 0.6203 - val_loss: 0.7875 - val_acc: 0.7749


In [33]:

y_true, y_pred = prediction_from_gen(gen=mobilenet_test_gen,
                                     steps=(1 + (test_data_df.shape[0] // batch_size)),
                                     model=model,
                                     dirname="mobilenet_test"
                                    )

In [34]:
# report test accuracy
test_accuracy = 100*np.sum(np.argmax(y_pred, axis=1)==np.argmax(y_true, axis=1))/len(y_true)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 76.9139%


In [35]:
# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = mobilenet.preprocess_input(paths_to_tensor(ttd_X1))
ttd_y_pred = model.predict(ttd_X2)

ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
ttd_y_true = ttd_data_df.y.as_matrix()

In [36]:
ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

TTD Test accuracy: 57.0295%
